### Намит Максим BD-21. Ранжирование документов по текстовой релевантности.

Код для соревнования достаточно хаотичен, в данном файле привожу некоторую его краткую 
выжимку (некоторые служебные части кода: запись в файл, чтение из файла чего-либо могут быть пропущены)

### 1. Обработка запросов. Спеллер от яндекса + нормализация.

In [ ]:
from pyaspeller import YandexSpeller
from tqdm.notebook import tqdm
from pyaspeller import Word
import pymorphy2
import gzip
from multiprocessing.dummy import  Queue
from multiprocessing import Pool
import os

In [ ]:
spellechecker = YandexSpeller(lang='ru', find_repeat_words=False, \
                              ignore_digits=True, max_requests=500)

In [ ]:
bad_q = []
with open("queries.tsv", 'r', encoding = 'utf-8') as f:
    for q in f:
        bad_q.append(q)

In [ ]:
fixes = []

Исправление запросов с помощью спеллера.

In [ ]:
for i, q in tqdm(enumerate(bad_q)):
    fix = spellechecker.spelled(q)
    for word in fix.split():
        word_check = Word(word)
        if not word_check.correct:
            fix = fix[:-1] + ' !!!!!!!!!!!!!!!!!' + '\n'
            break
    fixes.append(fix)

In [ ]:
with open("first_fixes.txt", 'w', encoding = 'utf-8') as f:
    for q in fixes:
        f.write(q)

Нормализация запросов.

In [ ]:
def to_normal(query):
    els = query.split('\t')
    token = word_tokenize(els[1])
    morph = pymorphy2.MorphAnalyzer()
    lemmas = [morph.parse(word)[0].normal_form for word in token]
    normal_query = els[0] + '\t' + ' '.join(lemmas) + '\n'
    return [normal_query]

In [ ]:
norm_queries = []

In [ ]:
with tqdm(total=len(queries)) as pbar:
    lock = pbar.get_lock()
    with Pool(processes=4) as pool:
        for i, el in enumerate(pool.imap_unordered(to_normal, queries)):
            with lock:
                norm_queries.extend(el)
                pbar.update(1)
pool.join()

### 2. Обработка данных : выгрузка данных, нормализация заголовков.

In [ ]:
from IPython.display import clear_output

По частям выгружаю файлы в txt формат (вроде брал по 100.000), затем архивирую их а исходные txt удаляю, иначе не хватало места на диске.

In [ ]:
with gzip.open('docs.tsv.gz', 'rb') as f:
    i = 0
    line = f.readline()
    while (i < 107045):
        line=f.readline()
        i += 1
        if i % 10000 == 0:
            clear_output()
            print('Doc i =', i, ' ALL = 582166')
    while (line):
        els = line.decode('utf8', 'ignore').split('\t')
        file = open('docs/'+els[0]+'.txt', "w")
        file.write(els[1] + '\n\t' + els[2])
        file.close()
        line=f.readline()
        i+=1
        if i > 107045:
            break
        if i % 1000 == 0:
            clear_output()
            print('Doc i =', i, ' ALL = 582166')

In [ ]:
def process(i):
    file = open('docs/'+str(i)+'.txt', "r")
    content = file.read()
    file.close()
    os.remove('docs/'+str(i)+'.txt')
    byte_data = (content).encode('utf-8')
    with gzip.open('docs/'+str(i)+'.txt.gz', "wb") as file:
        file.write(byte_data)

In [ ]:
with tqdm(total=len(queue)) as pbar:
    lock = pbar.get_lock()
    with Pool(processes=4) as pool:
        for i, _ in enumerate(pool.imap_unordered(process, queue)):
            with lock:
                pbar.update(1)
pool.join()

Выгружаем номера нужных документов.

In [ ]:
queries_doc = {}
with open("sample.csv", 'r') as f:
    f.readline()
    for line in tqdm(f.readlines()):
        split = line.split(',')
        if int(split[0]) not in queries_doc.keys():
            queries_doc[int(split[0])]=[int(split[1])]
        else:
            queries_doc[int(split[0])].append(int(split[1]))
for i in queries_doc:
    queries_doc[i] = sorted(queries_doc[i])
docs = set()
for key in queries_doc:
    for doc in queries_doc[key]:
        docs.add(doc)

Выгружаем нужные заголовки для их последующей нормализации.

In [ ]:
def get_titles(i):
    temp_titles = []
    temp_bad = []
    temp_bad_len = []
    with gzip.open('docs/'+str(i)+'.txt.gz', "r") as file:
        lines = file.read()
        els = lines.split(b'\t')
        if (len(els) < 2):
            temp_bad.append(i)
        title = str(i) + '\t' + els[0][:-1].decode('utf8', 'ignore').lower()
        if (len(title) > 500):
            temp_bad_len.append(i)
        temp_titles.append(title)
    return temp_titles, temp_bad, temp_bad_len

In [ ]:
titles = []
bad = []
bad_len = []

In [ ]:
with tqdm(total=len(queue)) as pbar:
    lock = pbar.get_lock()
    with Pool(processes=4) as pool:
        for i, el in enumerate(pool.imap_unordered(get_titles, queue)):
            with lock:
                titles.extend(el[0])
                bad.extend(el[1])
                bad_len.extend(el[2])
                pbar.update(1)
pool.join()

Нормализация заголовков.

In [ ]:
def to_normal(title):
    els = title.split('\t')
    token = word_tokenize(els[1])
    morph = pymorphy2.MorphAnalyzer()
    lemmas = [morph.parse(word)[0].normal_form for word in token]
    normal_title = els[0] + '\t' + ' '.join(lemmas) + '\n'
    return [normal_title]

In [ ]:
norm_queries = []

In [ ]:
with tqdm(total=len(queries)) as pbar:
    lock = pbar.get_lock()
    with Pool(processes=4) as pool:
        for i, el in enumerate(pool.imap_unordered(to_normal, queries)):
            with lock:
                norm_queries.extend(el)
                pbar.update(1)
pool.join()

In [ ]:
doc_titles_norm = dict()
for title in norm_titles:
    els = title.split('\t')
    doc_titles_norm[int(els[0])] = els[1]

In [ ]:
with open('titles_norm_1.txt', 'w') as f:
    for i in sorted(doc_titles_norm.keys()):
        f.write(str(i) + '\t' + doc_titles_norm[i])

### 3. Модель

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text

In [ ]:
queries_doc = {}
with open("sample.csv", 'r') as f:
    f.readline()
    for line in tqdm(f.readlines()):
        split = line.split(',')
        if int(split[0]) not in queries_doc.keys():
            queries_doc[int(split[0])]=[int(split[1])]
        else:
            queries_doc[int(split[0])].append(int(split[1]))
for i in queries_doc:
    queries_doc[i] = sorted(queries_doc[i])

In [ ]:
queries = []
# здесь также мог быть другой файл, в котором содержались не нормализованные запросы,
# в итоге лучший результат был получен с помощью не нормализованных запросов и заголовков
with open("norm_queries.txt", 'r', encoding = 'utf-8') as f:
    lines = f.readlines()
    for line in lines:
        queries.append(line)

In [ ]:
doc_queries = {}
for query in queries:
    els = query.split('\t')
    doc_queries[int(els[0])] = els[1][:-1]

In [ ]:
# если используем нормализованные заголовки их надо прочитать
norm_titles = {}
with open("titles_norm_1.txt", 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        els = line.split('\t')
        norm_titles[int(els[0])] = els[1]

Модель на заголовках - "грубый" результат.

In [ ]:
for i in tqdm(sorted(queries_doc.keys())):
    try:
        titles = []
        texts = []
        nums = []
        nums_titles = {}
        size = 0
        for doc in queries_doc[i]:
            # опять же можно использовать как нормализованные так и не нормализованные

            #with gzip.open('docs/'+str(doc)+'.txt.gz', "r") as file:
            #    text = file.read().split(b'\t')
            #    if len(text) < 2:
            #        continue
            #    title = text[0].decode('utf8', 'ignore')
            title = norm_titles[doc][:-1]
            if len(title) > 1000:
                title = title[:1000]
            titles.append(title)
            nums.append(doc)
        question_embeddings = module.signatures['question_encoder'](
            tf.constant([queries[i].split('\t')[1].lower()])
        )
        encodings = module.signatures['response_encoder'](
            input=tf.constant(titles),
            context=tf.constant(titles)
        )
        scores = np.inner(question_embeddings['outputs'], encodings['outputs'])
        tmp = {}
        for j in range(len(titles)):
            tmp[nums[j]] = scores[0, j]
        result[i] = sorted(tmp.items(), key = lambda x: x[1], reverse=True)
    except:
        bad.append(i)

In [ ]:
k = 0
with open('baseline_norm_titles.csv','w') as f:
    f.write('QueryId,DocumentId\n')
    for i in sorted(result.keys()):
        temp = result[i][:10]
        if (len(temp) != 10):
            print('!!!!!', i)
        for item in temp:
            f.write('{},{}\n'.format(i,item[0]))
            k += 1
print(k)

Добавляем контекст.

In [ ]:
with open('rough_result.txt', 'w') as f:
    for i in result:
        for j in result[i]:
            f.write(str(i) + ',' + str(j[0]) + '\n')

In [ ]:
result_30 = {}
with open("rough_result.txt", 'r') as f:
    for line in tqdm(f.readlines()):
        split = line.split(',')
        if int(split[0]) not in result_30.keys():
            result_30[int(split[0])]=[int(split[1])]
        else:
            result_30[int(split[0])].append(int(split[1]))

По 500-700 запросов для надежности (исполнение иногда падало). Лучше всего сработало именно на 30 документах. То есть имеем результат ранжирования на заголовках и переранжируем его с учетом документов.

In [ ]:
for i in tqdm(sorted(result_30.keys())[0:500]):
    try:
        titles = []
        texts = []
        nums = []
        nums_titles = {}
        size = 0
        for doc in result_30[i]:
            with gzip.open('docs/'+str(doc)+'.txt.gz', "r") as file:
                els = file.read().split(b'\t')
                if len(els) < 2:
                    continue
                title = els[0].decode('utf8', 'ignore')
                if len(title) > 500:
                    title = title[0:500]
                text = els[1].decode('utf8', 'ignore')
            titles.append(title[:-1].lower())
            texts.append(text.lower())
            nums.append(doc)
            nums_titles[doc] = title[:-1].lower()
        question_embeddings = module.signatures['question_encoder'](
            tf.constant([queries[i].split('\t')[1].lower()])
        )
        encodings = module.signatures['response_encoder'](
            input=tf.constant(titles),
            context=tf.constant(texts)
        )
        scores = np.inner(question_embeddings['outputs'], encodings['outputs'])
        tmp = {}
        for j in range(len(titles)):
            tmp[nums[j]] = scores[0, j]
        result[i] = sorted(tmp.items(), key = lambda x: x[1], reverse=True)
        with open('sub.txt', 'a') as f:
            f.write(str(i) + '\t ')
            for j in result[i]:
                f.write(str(j[0]) + ' ')
            f.write('\n')
    except:
        bad.append(i)

Пробовал увеличивать число документов - не помогло (возможно криво сделал, потому что разбил каждый запрос на 3 части чтобы влезло в RAM)

In [ ]:
for i in tqdm(sorted(result_30.keys())[314:]):
    tmp = {}
    for step in (range(3)):     
        titles = []
        texts = []
        nums = []
        nums_titles = {}
        size = 0
        start = int(step * len(result_30[i]) / 3)
        end   = int((step + 1) * len(result_30[i]) / 3 if step != 2 else -1)
        if end != -1:
            docs_part = result_30[i][start:end]
        else:
            docs_part = result_30[i][start:]
        for doc in docs_part:
            with gzip.open('docs/'+str(doc)+'.txt.gz', "r") as file:
                els = file.read().split(b'\t')
                if len(els) < 2:
                    continue
                title = els[0].decode('utf8', 'ignore')
                if len(title) > 500:
                    title = title[0:500]
                text = els[1].decode('utf8', 'ignore')
            titles.append(title[:-1].lower())
            texts.append(text.lower())
            nums.append(doc)
            nums_titles[doc] = title[:-1].lower()
        question_embeddings = module.signatures['question_encoder'](
            tf.constant([queries[i].split('\t')[1].lower()])
        )
        encodings = module.signatures['response_encoder'](
            input=tf.constant(titles),
            context=tf.constant(texts)
        )
        scores = np.inner(question_embeddings['outputs'], encodings['outputs'])
        for j in range(len(titles)):
            tmp[nums[j]] = scores[0, j]
    result[i] = sorted(tmp.items(), key = lambda x: x[1], reverse=True)
    with open('final.txt', 'a') as f:
        f.write(str(i) + '\t ')
        for j in result[i]:
            f.write(str(j[0]) + ' ')
        f.write('\n')